In [1]:
from pyspark.sql import types as T
from pyspark.sql import functions as F 
from pyspark.sql import DataFrame
from pyspark.sql.functions import * 
from pyspark.sql.window import Window
import sys

# from pyspark.sql.types import IntegerType
# from pyspark.sql.functions import first

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
36,application_1588279895888_0013,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Table 1A+1B+2 
registrations_raw = spark.sql("""
    select 
        UID.id as user_id
        , to_date(dcreated) as reg_date
        , passwordset
        , subscriptionid
        , to_date(subscriptiondate) as sub_date
        , isactive
        , manualunsubscribe
        , to_date(logdatetime) as log_date
        , logaction
        , count(logdatetime) as n_events
    from stepstonede.userlogin UID
    left join stepstonede.subscription SUB
    on UID.id = SUB.userlogin_id
    left join stepstonede.logmystst LOG
    on 
        UID.id = LOG.userloginid 
        and logaction in ("MyStSt,AccountValidation","AUM,AccountValidation", "AUM,confirmation", "IJM, subscription", "SIM_LastOpen", "IJM, autodeactivate", "IJM, autoactivate", 
            "IJM, Manual Deactivate", "IJM, activation", "IJM, unsubscription", "JA,activation") --- added "JA,activation" and "AUM,confirmation" 
    where
        dcreated >= '2020-01-01' 
    group by
        UID.id 
        , reg_date
        , passwordset
        , subscriptionid
        , sub_date
        , isactive
        , manualunsubscribe
        , log_date
        , logaction
    order by reg_date, log_date""")


#"JA,activation" added to logaction list

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# New: these are the ratios i spoke about - count of distinct user id values
registrations_raw.agg(F.countDistinct("user_id").alias("registrations"),
                      F.countDistinct(F.when(F.col("logaction").isin(["MyStSt,AccountValidation","AUM,AccountValidation","JA, activation","AUM,confirmation"]), F.col("user_id"))).alias("accountactivation"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, subscription"]), F.col("user_id"))).alias("IJMSubscription"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, unsubscription"]), F.col("user_id"))).alias("IJMunsubscribe"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, autoactivate"]), F.col("user_id"))).alias("IJMautoactivation"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, autodeactivate"]), F.col("user_id"))).alias("IJMautodeactivation"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, Manual Deactivate"]), F.col("user_id"))).alias("IJMManualDeactivation"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, activation"]), F.col("user_id"))).alias("IJMactivation")).show()
# from this we get ratios to validate:
# - Account activations = 253000/400225 = 63% this seems to be OK
# - IJMSubscriptions = 232952 / 400225 = this seems about right + check against subscription table
# - IJMunsubscribe = 22 this number is much too low! This log doesnt seem to be written correctly
# - IJMautoactivation = roughly 1/4 of registrations: seems very low
# - IJMautodeactivation - similar magnitude as IJMautoactivation - does seem to make sense
# - IJMactivation - does not seem to be used anymore

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----------------+---------------+--------------+-----------------+-------------------+---------------------+-------------+
|registrations|accountactivation|IJMSubscription|IJMunsubscribe|IJMautoactivation|IJMautodeactivation|IJMManualDeactivation|IJMactivation|
+-------------+-----------------+---------------+--------------+-----------------+-------------------+---------------------+-------------+
|       403794|           254149|         236350|            22|            94120|              94341|                    0|            0|
+-------------+-----------------+---------------+--------------+-----------------+-------------------+---------------------+-------------+

In [4]:
# here with changed logic for IJM subscription and IJMunsubscribe
registrations_raw.agg(F.countDistinct("user_id").alias("registrations"),
                      F.countDistinct(F.when(F.col("logaction").isin(["MyStSt,AccountValidation","AUM,AccountValidation","JA, activation","AUM,confirmation"]), F.col("user_id"))).alias("accountactivation"),
                      F.countDistinct(F.when(F.col("subscriptionid").isNotNull(), F.col("user_id"))).alias("IJMSubscription"),
                      F.countDistinct(F.when(F.col("manualunsubscribe") == True, F.col("user_id"))).alias("IJMunsubscribe"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, autoactivate"]), F.col("user_id"))).alias("IJMautoactivation"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, autodeactivate"]), F.col("user_id"))).alias("IJMautodeactivation"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, Manual Deactivate"]), F.col("user_id"))).alias("IJMManualDeactivation"),
                      F.countDistinct(F.when(F.col("logaction").isin(["IJM, activation"]), F.col("user_id"))).alias("IJMactivation")).show()
# ijm subscription yields similar number as above - so both logmystst and subscription table seem to be ok to use
# ijm unsubscription captured in subscription table - but not sure if we can find a timestamp for this? we know from another analysis that majority of unsubscribes are done on day 1 of receiving an email, 
# so we could use subscription date as the unsubscribe date

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----------------+---------------+--------------+-----------------+-------------------+---------------------+-------------+
|registrations|accountactivation|IJMSubscription|IJMunsubscribe|IJMautoactivation|IJMautodeactivation|IJMManualDeactivation|IJMactivation|
+-------------+-----------------+---------------+--------------+-----------------+-------------------+---------------------+-------------+
|       403794|           254149|         236665|         41718|            94120|              94341|                    0|            0|
+-------------+-----------------+---------------+--------------+-----------------+-------------------+---------------------+-------------+

In [5]:
# Conclusions:
# we assume account validation is stored correctly in logactions ["MyStSt,AccountValidation","AUM,AccountValidation","JA, activation","AUM,confirmation"]
# we take the IJM subscription information from logactions
# we take the IJM unsubscribe action from the subscription table and assume the unsubscribe happens on the day of the subscription
# we include ijmautoactivation and ijmdeactivation in the analysis

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#UPDATED!!!
# Table 2: validated accounts
registration_df = registrations_raw.select("user_id", "reg_date").distinct().\
    join(registrations_raw.select("user_id", "logaction", "log_date"). \
        filter(F.col("logaction").isin(["MyStSt,AccountValidation","AUM,AccountValidation","JA, activation","AUM,confirmation"])). \
        groupBy("user_id"). \
        agg(F.min("log_date").alias("AccountValidation")), "user_id", "left")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
#CHECK!
registration_df.agg(F.countDistinct("user_id").alias("regs"),
                    F.countDistinct(F.when(F.col("AccountValidation").isNotNull(), F.col("user_id"))).alias("validated_accounts")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------------------+
|  regs|validated_accounts|
+------+------------------+
|403794|            254149|
+------+------------------+

In [8]:
# UPDATED!!!!
# Table2A: Validated accounts & considered_dates
# create DF with all dates from first day to last day per user
# user_df =
considered_date_df = registrations_raw.select("user_id", "reg_date"). \
    distinct(). \
    withColumn("first_day", F.to_date(F.col("reg_date"))). \
    withColumn("last_day", F.to_date(F.lit(registrations_raw.select(F.max(F.col("reg_date")).alias("max_date")).collect()[0][0]))). \
    withColumn("diffDays",  F.datediff("last_day", "first_day")). \
    withColumn("repeat", F.expr("split(repeat(',', diffDays), ',')")).\
    select("*", F.posexplode("repeat").alias("daycount", "val")).\
    drop("repeat", "val", "diffDays").\
    withColumn("considered_date", F.expr("date_add(first_day, daycount)")).\
    select("user_id", "considered_date")

validated_considered_df = registration_df.join(considered_date_df, "user_id", "left").\
    withColumn("validated_account", F.when(F.col("AccountValidation")<=F.col("considered_date"), F.lit(1)).otherwise(F.lit(0))).\
    select("user_id", "reg_date", "considered_date", "validated_account")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# CHECK!
validated_considered_df.groupBy("considered_date").\
    agg(F.countDistinct("user_id").alias("regs"),
        F.countDistinct(F.when(F.col("validated_account") == 1, F.col("user_id"))).alias("validated_accounts")).\
    sort(F.col("considered_date")).show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----+------------------+
|considered_date| regs|validated_accounts|
+---------------+-----+------------------+
|     2020-01-01| 1450|               893|
|     2020-01-02| 4699|              3039|
|     2020-01-03| 7957|              5358|
|     2020-01-04|10540|              7263|
|     2020-01-05|13566|              9407|
|     2020-01-06|17681|             12293|
|     2020-01-07|21822|             15321|
|     2020-01-08|25985|             18346|
|     2020-01-09|29518|             20959|
|     2020-01-10|32367|             23135|
|     2020-01-11|34610|             24830|
|     2020-01-12|37812|             27221|
|     2020-01-13|41954|             30307|
|     2020-01-14|46151|             33427|
|     2020-01-15|50003|             36267|
|     2020-01-16|53805|             39122|
|     2020-01-17|56681|             41234|
|     2020-01-18|58996|             42842|
|     2020-01-19|61879|             44897|
|     2020-01-20|66031|             47914|
+----------

In [10]:
#UPDATED!!!
# Table2B: IJM subscription 
IJM_subscription_df = registrations_raw. \
    filter(F.col("logaction").isin(["IJM, subscription"])). \
    select("user_id", "log_date", "n_events").\
    withColumn("n_events", F.when(F.col("n_events").isNull(), F.lit(0)).otherwise(F.col("n_events")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#UPDATED!!!
#Table3.1 (considered_subscribed_df): Join Table2a(validated_considered_df) and Table2B(IJM_subscription_df) 
#on User ID and considered_date = subscription date
#join type: left (please confirm)

#register intermidiate tables to be able to use sql
validated_considered_df.registerTempTable("validated_considered_df")
IJM_subscription_df.registerTempTable("IJM_subscription_df")

#join Table2A (validated_considered_df) to Table2B (IJM_subscription_df) 
#on 'user_id' and considered_date = subscription date

considered_subscribed_df1 = spark.sql("""
    select *
    from validated_considered_df as df1
    left join IJM_subscription_df as df2
    on df1.user_id = df2.user_id
    and df1.considered_date = df2.log_date
    """
)

considered_subscribed_df = considered_subscribed_df1.select(validated_considered_df.user_id,
                                                            validated_considered_df.reg_date,
                                                            'validated_account', 
                                                            'considered_date', 
                                                            col('n_events').alias('sub_events')).distinct()

considered_subscribed_df = considered_subscribed_df.fillna({'sub_events':'0'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# CHECK!
considered_subscribed_df.groupBy("considered_date").\
    agg(F.countDistinct("user_id").alias("regs"),
        F.countDistinct(F.when(F.col("validated_account") == 1, F.col("user_id"))).alias("validated_accounts"),
        F.countDistinct(F.when(F.col("sub_events") >= 1, F.col("user_id"))).alias("new_subscriptions")).\
    sort(F.col("considered_date")).show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----+------------------+-----------------+
|considered_date| regs|validated_accounts|new_subscriptions|
+---------------+-----+------------------+-----------------+
|     2020-01-01| 1450|               893|              674|
|     2020-01-02| 4699|              3039|             1735|
|     2020-01-03| 7957|              5358|             1907|
|     2020-01-04|10540|              7263|             1508|
|     2020-01-05|13566|              9407|             1699|
|     2020-01-06|17681|             12293|             2348|
|     2020-01-07|21822|             15321|             2539|
|     2020-01-08|25985|             18346|             2581|
|     2020-01-09|29518|             20959|             2303|
|     2020-01-10|32367|             23135|             1873|
|     2020-01-11|34610|             24830|             1441|
|     2020-01-12|37812|             27221|             2022|
|     2020-01-13|41954|             30307|             2756|
|     2020-01-14|46151| 

In [13]:
# UPDATED!!!
#Table2C: (IJM_unsubscription_df)
IJM_unsubscription_df = registrations_raw. \
    filter(F.col("manualunsubscribe") == True). \
    select("user_id", "manualunsubscribe").distinct()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# UPDATED!!! we use anti_join, since we dont have an unsubscribe timestamp, that means all users who ever unscrubscribed are removed from the data!
#Table3.2 (considered_unsubscribed_df): Join Table3.1(considered_subscribed_df) and Table2C(IJM_unsubscription_df) 
#on User ID 
#subscribe the tables to be able to use sql to join them
considered_subscribed_df.registerTempTable("considered_subscribed_df")
IJM_unsubscription_df.registerTempTable("IJM_unsubscription_df")

considered_unsubscribed_df = considered_subscribed_df.join(IJM_unsubscription_df, "user_id", "left_anti").\
    withColumn("sub_events", F.when(F.col("sub_events")>=1, F.lit(1)).otherwise(F.col("sub_events"))).\
    select("user_id", "reg_date", "validated_account", "considered_date", "sub_events")

considered_unsubscribed_df = considered_unsubscribed_df.fillna({'sub_events':'0'})


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# UPDATED!!!!
#Table2D: IJM_autoactivate_df
IJM_autoactivate_df = registrations_raw. \
    filter(F.col("logaction").isin(["IJM, autoactivate"])). \
    select("user_id", "log_date", "n_events").distinct()

IJM_autoactivate_df = IJM_autoactivate_df.fillna({'n_events':'0'})

#IJM_autoactivate_df.show(n=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# UPDATED!!!
# Table3.3 (considered_autoactivated_df): Join Table3.2(considered_unsubscribed_df) and Table2D(IJM_autoactivate_df) 
# on User ID and considered_date = autoactivation date
    
#subscribe the tables to be able to use sql to join them
considered_unsubscribed_df.registerTempTable("considered_unsubscribed_df")
IJM_autoactivate_df.registerTempTable("IJM_autoactivate_df")

considered_autoactivated_df = spark.sql("""
    select df1.user_id, reg_date, validated_account, considered_date, sub_events, n_events as autoactivated_events
    from considered_unsubscribed_df as df1
    left join IJM_autoactivate_df as df2
    on df1.user_id = df2.user_id
    and df1.considered_date = df2.log_date""")

considered_autoactivated_df = considered_autoactivated_df.fillna({'autoactivated_events':'0'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# UPDATED!!!
#Table2E: IJM_autodeactivate_df
IJM_autodeactivate_df = registrations_raw. \
    filter(F.col("logaction").isin(["IJM, autodeactivate"])). \
    select("user_id","log_date", "n_events")

IJM_autodeactivate_df = IJM_autodeactivate_df.fillna({'n_events':'0'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# UPDATED!!!
# Table3.4 (considered_autodeactivated_df): Join Table3.3(considered_autoactivated_df) and Table2E(IJM_deautoactivate_df)
# on User ID and considered_date = autodeactivate date
# join type: left (please confirm)  
    
#subscribe the tables to be able to use sql to join them
considered_autoactivated_df.registerTempTable("considered_autoactivated_df")
IJM_autodeactivate_df.registerTempTable("IJM_autodeactivate_df")

email_log_df = spark.sql("""
    select df1.user_id, reg_date, validated_account, considered_date, sub_events, autoactivated_events, n_events as autodeactivate_events
    from considered_autoactivated_df as df1
    left join IJM_autodeactivate_df as df2
    on df1.user_id = df2.user_id
    and df1.considered_date = df2.log_date
"""
)

email_log_df = email_log_df.fillna({'autodeactivate_events':'0'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# UPDATED!!!
#email_log_df: Add the cumulative sum of 'sub_sum'
email_log_df = email_log_df.select("user_id", "reg_date", "considered_date", "validated_account", "sub_events", "autoactivated_events", "autodeactivate_events"). \
    distinct(). \
    withColumn('sub_sum', F.sum(email_log_df.sub_events).over(Window.partitionBy('user_id'). \
        orderBy('considered_date').rowsBetween(-sys.maxsize, 0))) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# UPDATED!!!
#email_log_df: Add the cumulative sum of 'autoactivated_sum'
email_log_df = email_log_df.select("user_id", "reg_date", "considered_date","validated_account", "sub_events", "autoactivated_events", "autodeactivate_events", "sub_sum"). \
    distinct(). \
    withColumn('autoactivated_sum', F.sum(email_log_df.autoactivated_events).over(Window.partitionBy('user_id'). \
        orderBy('considered_date').rowsBetween(-sys.maxsize, 0)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# UPDATED!!!
#email_log_df: Add the cumulative sum of 'autodeactivate_sum'
email_log_df = email_log_df.select("user_id", "reg_date", "considered_date","validated_account", "sub_events", "autoactivated_events", "autodeactivate_events", "sub_sum", "autoactivated_sum"). \
    distinct(). \
    withColumn('autodeactivate_sum', F.sum(email_log_df.autodeactivate_events).over(Window.partitionBy('user_id'). \
        orderBy('considered_date').rowsBetween(-sys.maxsize, 0)))

#email_log_df.show(n=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# UPDATED!!!
email_log_df = email_log_df.withColumn("sub_state", F.when(F.col("sub_sum")>=1, F.lit(1)).otherwise(F.lit(0))).\
    withColumn('autoactivate_state', F.when((col("autoactivated_sum")-col("autodeactivate_sum")) <= 0,F.lit(0)).otherwise(F.lit(1)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
email_log_df.groupBy("considered_date").\
    agg(F.countDistinct("user_id").alias("registrations"),
        F.countDistinct(F.when(F.col("validated_account") >=1, F.col("user_id"))).alias("validated_accounts"),
        F.countDistinct(F.when(F.col("sub_state") >=1, F.col("user_id"))).alias("subscriptions"),
        F.countDistinct(F.when(F.col("autoactivated_sum") >=1, F.col("user_id"))).alias("activated_subscriptions_v0"),
        F.countDistinct(F.when(F.col("autoactivate_state") >=1, F.col("user_id"))).alias("activated_subscriptions")).\
    sort("considered_date").show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------------+------------------+-------------+--------------------------+-----------------------+
|considered_date|registrations|validated_accounts|subscriptions|activated_subscriptions_v0|activated_subscriptions|
+---------------+-------------+------------------+-------------+--------------------------+-----------------------+
|     2020-01-01|         1260|               738|          519|                         0|                      0|
|     2020-01-02|         4067|              2486|         1833|                         0|                      0|
|     2020-01-03|         6823|              4361|         3262|                         0|                      0|
|     2020-01-04|         9046|              5938|         4409|                         1|                      1|
|     2020-01-05|        11652|              7686|         5669|                         1|                      1|
|     2020-01-06|        15148|             10005|         7382|        